<a href="https://colab.research.google.com/github/Abhish2702/BTP-Depression-Detection-/blob/main/btp_using_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
d_tweets_dataset=pd.read_csv("/content/clean_d_tweets.csv")
non_d_tweets_dataset=pd.read_csv("/content/clean_non_d_tweets.csv")
d_tweets_dataset.tweet = d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.tweet = non_d_tweets_dataset.tweet.fillna('')

In [14]:
d_tweets=d_tweets_dataset['tweet']
d_tweets.head()
d_tweets.shape

(3082,)

In [4]:
non_d_tweets=non_d_tweets_dataset['tweet']
non_d_tweets.head()

0                           touch pass be the new asmr
1    12 years ago i call my dad and he give me some...
2    there be 6500 languages yet i choose to only s...
3    hey everyone ask for a raise today i be tell y...
4    i invest in 47 startups in a year here be how ...
Name: tweet, dtype: object

In [5]:
non_d_tweets.shape

(2505,)

In [32]:
y1=np.ones((3082,1),dtype="int")

In [33]:
y1.shape

(3082, 1)

In [34]:
y2=np.zeros((2505,1),dtype="int")

In [35]:
y2.shape

(2505, 1)

In [36]:
y=np.concatenate((y1,y2),axis=0)

In [37]:
y

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [38]:
res=pd.DataFrame(y)

In [39]:
res

,0
0,1
1,1
2,1
3,1
4,1
...,...
5582,0
5583,0
5584,0
5585,0


In [40]:
res.shape

(5587, 1)

In [41]:
txt=pd.concat([d_tweets,non_d_tweets],axis=0)

In [42]:
txt.head()

0    the real reason why you be sad you be attach t...
1        my biggest problem be overthinking everything
2    the worst sadness be the sadness you have teac...
3    i cannot make you understand i cannot make any...
4    i do not think anyone really understand how ti...
Name: tweet, dtype: object

In [43]:
txt.tail()

2500    wake up with awful anxiety i guess its a tea d...
2501            sorry i dress as giselle and not toms son
2502    i be go to be a computer for halloween so my b...
2503                the sanderson sisters only drink fire
2504     i send my feet pic and he still will not play us
Name: tweet, dtype: object

In [47]:
txt.shape
txt=txt.reset_index(drop="true")

In [52]:
df=pd.concat([txt,res],axis=1,join="inner")

In [53]:
df.columns=["text","result"]


In [54]:
df

,text,result
0,the real reason why you be sad you be attach t...,1
1,my biggest problem be overthinking everything,1
2,the worst sadness be the sadness you have teac...,1
3,i cannot make you understand i cannot make any...,1
4,i do not think anyone really understand how ti...,1
...,...,...
5582,wake up with awful anxiety i guess its a tea d...,0
5583,sorry i dress as giselle and not toms son,0
5584,i be go to be a computer for halloween so my b...,0
5585,the sanderson sisters only drink fire,0


In [56]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your dataset
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text data
tokenized_text = [word_tokenize(text.lower()) for text in df['text']]

# Step 2: Train Word2Vec Model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Step 3: Generate Word Embeddings
word_vectors = word2vec_model.wv

# Step 4: Convert Text to Word Embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)
X_sequences = tokenizer.texts_to_sequences(tokenized_text)
X_padded = pad_sequences(X_sequences, maxlen=100)

# Step 5: Prepare the Target Variable
y = df['result'].values

# Step 6: Split Data into Train and Test Sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the LSTM-RNN Model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=100))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 8: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 9: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Step 10: Make Predictions
# predictions = model.predict_classes(X_test)

# You can use predictions for further analysis or evaluation.


Epoch 1/10
140/140 [==============================] - 46s 297ms/step - loss: 0.4706 - accuracy: 0.7691 - val_loss: 0.3253 - val_accuracy: 0.8497
Epoch 2/10
140/140 [==============================] - 40s 289ms/step - loss: 0.2333 - accuracy: 0.9045 - val_loss: 0.2856 - val_accuracy: 0.8730
Epoch 3/10
140/140 [==============================] - 41s 292ms/step - loss: 0.1322 - accuracy: 0.9535 - val_loss: 0.2788 - val_accuracy: 0.8775
Epoch 4/10
140/140 [==============================] - 40s 288ms/step - loss: 0.0878 - accuracy: 0.9693 - val_loss: 0.3458 - val_accuracy: 0.8676
Epoch 5/10
140/140 [==============================] - 40s 289ms/step - loss: 0.1004 - accuracy: 0.9702 - val_loss: 0.3718 - val_accuracy: 0.8810
Epoch 6/10
140/140 [==============================] - 40s 289ms/step - loss: 0.0537 - accuracy: 0.9817 - val_loss: 0.4030 - val_accuracy: 0.8792
Epoch 7/10
140/140 [==============================] - 42s 298ms/step - loss: 0.0352 - accuracy: 0.9866 - val_loss: 0.4061 - val_ac

AttributeError: ignored